#### Notebook que carrega um valor estimado para os trechos de uma ferrovia/rodovia, associa esses trechos aos polígonos correspondetes na malha e então compara com o valor do mesmo indicador informado em uma divisão geométrica diferente.

In [ ]:
! pip install sqlalchemy
! pip install psycopg2
! pip install GeoAlchemy2
! pip install scikit-learn
! pip install bokeh
! pip install seaborn

In [52]:
from glob import glob
import geopandas as gpd
import pandas as pd
import os
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

##### Arquivos a processar, nome do arquivo de saída.

In [73]:
FERROVIAS_FNAME = r'D:\Atrium\Projects\AdaptaBrasil\Data\Adaptavias\Ferrovias\Ferrovias.shp'
ARQUIVO_1 = r'D:\Atrium\Projects\AdaptaBrasil\Data\Adaptavias\02_FERROVIAS_Adriano\04_BASE_DE_DADOS\B_RIS_DESLIZAMENTO\Nivel_2\AdVIA_FER_DESL_N2_PB_1981_2000_ln_EXPOSICAO.shp'
ARQUIVO_2 = r'D:\Atrium\Projects\AdaptaBrasil\Data\Adaptavias\Solucao_Ind_exposicao_ferrovias.xlsx'
nome_indicador = os.path.basename(ARQUIVO_1).split('.')[0]
ARQUIVO_SAIDA = fr'D:\Atrium\Projects\AdaptaBrasil\Data\Adaptavias\{nome_indicador}.png'

##### Carrega e ajusta arquivos.

In [54]:
ferrovias = gpd.read_file(FERROVIAS_FNAME)

In [55]:
gdf = gpd.read_file(ARQUIVO_1)

In [56]:
df = pd.read_excel(ARQUIVO_2)

In [57]:
df.columns = ['objectid','valores']
df.head()

,objectid,valores
0,7472,0.01
1,7476,0.01
2,7482,0.01
3,7483,1.00
4,7487,0.01


In [ ]:
gdf.head()

In [42]:
print(len(df))
print(len(gdf))
print(len(ferrovias))

1428
2909
1428


##### Primeiro junta indicador com a ferrovia, obtendo as geometrias.

In [58]:
ferrovias = ferrovias.merge(df, on='objectid', how='inner')

In [59]:
print(len(ferrovias))
print(type(ferrovias))

1428
<class 'geopandas.geodataframe.GeoDataFrame'>


#### Depois, junta indicador+ferrovias com o valor original do indicador.

In [60]:
intersecao = gpd.sjoin(gdf, ferrovias, how='inner', predicate='intersects')

In [61]:
print(len(intersecao))
intersecao.head()

3860


,ID,ID_1,PROJETO,MODO,IMP_INTER,CENARIO,PERIODO,DADO_BASE,TIP_SITUAC,BITOLA,...,sigla,sigla_coin,municipio,id,uf,label,nome,extensao,st_length_,valores
0,1,1.0,AdaptVIAS (2022),FERROVIÁRIO,DESLIZAMENTO - Exposição,Período Base,1981 - 2000,Ferrovias_MINFRA,Em Operação,Larga,...,EF-364,RMN,Alto Araguaia,1792,MT,None,Rumo Malha Norte - RMN,85.559378,0.791396,0.25
2403,2404,2404.0,AdaptVIAS (2022),FERROVIÁRIO,DESLIZAMENTO - Exposição,Período Base,1981 - 2000,Ferrovias_MINFRA,Em Operação,Larga,...,EF-364,RMN,Alto Araguaia,1792,MT,None,Rumo Malha Norte - RMN,85.559378,0.791396,0.25
2730,2731,2731.0,AdaptVIAS (2022),FERROVIÁRIO,DESLIZAMENTO - Exposição,Período Base,1981 - 2000,Ferrovias_MINFRA,Em Operação,Larga,...,EF-364,RMN,Alto Araguaia,1792,MT,None,Rumo Malha Norte - RMN,85.559378,0.791396,0.25
1,2,2.0,AdaptVIAS (2022),FERROVIÁRIO,DESLIZAMENTO - Exposição,Período Base,1981 - 2000,Ferrovias_MINFRA,Em Operação,Larga,...,EF-364,RMN,Chapadão do Sul,1811,MS,None,Rumo Malha Norte - RMN,57.422988,0.536180,0.25
2737,2738,2738.0,AdaptVIAS (2022),FERROVIÁRIO,DESLIZAMENTO - Exposição,Período Base,1981 - 2000,Ferrovias_MINFRA,Em Operação,Larga,...,EF-364,RMN,Chapadão do Sul,1811,MS,None,Rumo Malha Norte - RMN,57.422988,0.536180,0.25


##### Rotina que gera a matriz de confusão.

In [49]:
CONFUSION_BINS =   [0, 0.01, 0.25, 0.50, 0.75, 1]
CONFUSION_LABELS = ['0.00 a 0.01', '0.01 a 0.25', '0.25 a 0.50', '0.50 a 0.75', '0.75 a 1']

In [74]:
def matrizConfusao(classe_original, classe_ponderada, display_labels: list, indicador: str):
    cfm=confusion_matrix(classe_original, classe_ponderada, normalize='all', labels=display_labels).round(4)*100
    f, ax = plt.subplots(1, 1, figsize=(8, 6))
    sns.heatmap(cfm, annot=True, fmt='g', ax=ax) #annot=True to annotate cells, ftm='g' to disable scientific notation

    # labels, title and ticks
    ax.set_xlabel('Classe Original')
    ax.set_ylabel('Classe Ponderada')
    ax.set_title(f'Matriz de Confusão (%)\n{indicador}')
    ax.xaxis.set_ticklabels(display_labels)
    ax.yaxis.set_ticklabels(display_labels)
    plt.savefig(ARQUIVO_SAIDA)
    ax.remove()
    plt.clf()


##### Cria as variáveis categóricas.

In [63]:
intersecao['label_indicador'] = pd.cut(x=intersecao['CL_N_ORIG'], bins=CONFUSION_BINS,
                                       labels=CONFUSION_LABELS)
intersecao['label_estimativa'] = pd.cut(x=intersecao['valores'], bins=CONFUSION_BINS,
                                       labels=CONFUSION_LABELS)

In [64]:
intersecao.head()

,ID,ID_1,PROJETO,MODO,IMP_INTER,CENARIO,PERIODO,DADO_BASE,TIP_SITUAC,BITOLA,...,municipio,id,uf,label,nome,extensao,st_length_,valores,label_indicador,label_estimativa
0,1,1.0,AdaptVIAS (2022),FERROVIÁRIO,DESLIZAMENTO - Exposição,Período Base,1981 - 2000,Ferrovias_MINFRA,Em Operação,Larga,...,Alto Araguaia,1792,MT,None,Rumo Malha Norte - RMN,85.559378,0.791396,0.25,0.01 a 0.25,0.01 a 0.25
2403,2404,2404.0,AdaptVIAS (2022),FERROVIÁRIO,DESLIZAMENTO - Exposição,Período Base,1981 - 2000,Ferrovias_MINFRA,Em Operação,Larga,...,Alto Araguaia,1792,MT,None,Rumo Malha Norte - RMN,85.559378,0.791396,0.25,0.01 a 0.25,0.01 a 0.25
2730,2731,2731.0,AdaptVIAS (2022),FERROVIÁRIO,DESLIZAMENTO - Exposição,Período Base,1981 - 2000,Ferrovias_MINFRA,Em Operação,Larga,...,Alto Araguaia,1792,MT,None,Rumo Malha Norte - RMN,85.559378,0.791396,0.25,0.01 a 0.25,0.01 a 0.25
1,2,2.0,AdaptVIAS (2022),FERROVIÁRIO,DESLIZAMENTO - Exposição,Período Base,1981 - 2000,Ferrovias_MINFRA,Em Operação,Larga,...,Chapadão do Sul,1811,MS,None,Rumo Malha Norte - RMN,57.422988,0.536180,0.25,0.01 a 0.25,0.01 a 0.25
2737,2738,2738.0,AdaptVIAS (2022),FERROVIÁRIO,DESLIZAMENTO - Exposição,Período Base,1981 - 2000,Ferrovias_MINFRA,Em Operação,Larga,...,Chapadão do Sul,1811,MS,None,Rumo Malha Norte - RMN,57.422988,0.536180,0.25,0.01 a 0.25,0.01 a 0.25


##### Gera e salva a matriz de confusão.

In [75]:
matrizConfusao(intersecao['label_indicador'],
               intersecao['label_estimativa'],
               CONFUSION_LABELS,
               nome_indicador)

<Figure size 800x600 with 0 Axes>